# Locational Embeddings POC

This notebook demonstrates an experimental approach to embedding location -- specifically counties in the US

The experiment was not successful.  Please ignore.

In [3]:
!pip install geopy

In [40]:
!pip install sklearn

  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=27f81cb35dc7490829dc68d685cc716a029bb8fbd6c4fb1bda74f78fa8398499
  Stored in directory: c:\users\noahk\appdata\local\pip\cache\wheels\e4\7b\98\b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built sklearn


In [85]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import one_hot
import pandas as pd
from geopy import distance
from sklearn.model_selection import train_test_split


### Load county coordinates and create coordinate pairs

In [18]:
df = pd.read_csv('2021_Gaz_counties_national.txt', delimiter='\t')
df.rename(columns={'INTPTLONG                                                                                                               ': 'INTPTLONG'}, inplace=True)
df

,USPS,GEOID,ANSICODE,NAME,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,AL,1001,161526,Autauga County,1539634184,25674812,594.456,9.913,32.532237,-86.646440
1,AL,1003,161527,Baldwin County,4117656514,1132955729,1589.836,437.437,30.659218,-87.746067
2,AL,1005,161528,Barbour County,2292160149,50523213,885.008,19.507,31.870253,-85.405104
3,AL,1007,161529,Bibb County,1612188717,9572303,622.470,3.696,33.015893,-87.127148
4,AL,1009,161530,Blount County,1670259090,14860281,644.891,5.738,33.977358,-86.566440
...,...,...,...,...,...,...,...,...,...,...
3216,PR,72145,1804553,Vega Baja Municipio,118746309,57826361,45.848,22.327,18.455128,-66.397883
3217,PR,72147,1804554,Vieques Municipio,131541389,552192821,50.788,213.203,18.125418,-65.432474
3218,PR,72149,1804555,Villalba Municipio,92298569,3622640,35.637,1.399,18.130718,-66.472244
3219,PR,72151,1804556,Yabucoa Municipio,143005188,72592515,55.215,28.028,18.059858,-65.859871


In [21]:
# Filter to the FIPS (GEOID), Latitude, and Longitude (of centroid), then take the cartesian product.  This gives us all pairs
df = df[['GEOID', 'INTPTLAT', 'INTPTLONG' ]]
df = df.merge(df, how='cross')
df

,GEOID_x,INTPTLAT_x,INTPTLONG_x,GEOID_y,INTPTLAT_y,INTPTLONG_y
0,1001,32.532237,-86.646440,1001,32.532237,-86.646440
1,1001,32.532237,-86.646440,1003,30.659218,-87.746067
2,1001,32.532237,-86.646440,1005,31.870253,-85.405104
3,1001,32.532237,-86.646440,1007,33.015893,-87.127148
4,1001,32.532237,-86.646440,1009,33.977358,-86.566440
...,...,...,...,...,...,...
10374836,72153,18.085669,-66.857901,72145,18.455128,-66.397883
10374837,72153,18.085669,-66.857901,72147,18.125418,-65.432474
10374838,72153,18.085669,-66.857901,72149,18.130718,-66.472244
10374839,72153,18.085669,-66.857901,72151,18.059858,-65.859871


In [23]:
# Calculate the distance

def calc_distance(row):
    return distance.distance((row['INTPTLAT_x'], row['INTPTLONG_x']), (row['INTPTLAT_y'], row['INTPTLONG_y'])).km

df['distance'] = df.apply(lambda row: calc_distance(row), axis=1)
df

,GEOID_x,INTPTLAT_x,INTPTLONG_x,GEOID_y,INTPTLAT_y,INTPTLONG_y,distance
0,1001,32.532237,-86.646440,1001,32.532237,-86.646440,0.000000
1,1001,32.532237,-86.646440,1003,30.659218,-87.746067,232.421886
2,1001,32.532237,-86.646440,1005,31.870253,-85.405104,138.154792
3,1001,32.532237,-86.646440,1007,33.015893,-87.127148,70.038399
4,1001,32.532237,-86.646440,1009,33.977358,-86.566440,160.450208
...,...,...,...,...,...,...,...
10374836,72153,18.085669,-66.857901,72145,18.455128,-66.397883,63.548465
10374837,72153,18.085669,-66.857901,72147,18.125418,-65.432474,150.933513
10374838,72153,18.085669,-66.857901,72149,18.130718,-66.472244,41.121418
10374839,72153,18.085669,-66.857901,72151,18.059858,-65.859871,105.691401


In [24]:
df.to_pickle('distances.pkl')

In [34]:
df = pd.read_pickle('distances.pkl')

In [37]:
df['distance_meters'] = df['distance'] * 1000
dataset = df[['GEOID_x', 'GEOID_y', 'distance_meters']].astype('int32').to_numpy()
X = dataset[:,:2]
y = dataset[:,-1]
X, y

(array([[ 1001,  1001],
        [ 1001,  1003],
        [ 1001,  1005],
        ...,
        [72153, 72149],
        [72153, 72151],
        [72153, 72153]]),
 array([     0, 232421, 138154, ...,  41121, 105691,      0]))

In [89]:
# X_1 = tf.convert_to_tensor(X[:,0].reshape(-1,1))
# X_2 = tf.convert_to_tensor(X[:,1].reshape(-1,1))
# X_1 = tf.convert_to_tensor(X[:,0])
# X_2 = tf.convert_to_tensor(X[:,1])
values = set(X[:,0])
one_hot_1 = [one_hot(str(x), len(values)) for x in X[:,0] ]
one_hot_2 = [one_hot(str(x), len(values)) for x in X[:,1] ]
y_str = [str(y_) for y_ in y]
embedding_dim = 2
input_1 = keras.layers.Input(shape=[len(values)])
input_2 = keras.layers.Input(shape=[len(values)])
embedding_1 = keras.layers.Embedding(input_dim=len(values), output_dim=embedding_dim)(input_1)
embedding_2 = keras.layers.Embedding(input_dim=len(values), output_dim=embedding_dim)(input_2)
encoded_inputs = keras.layers.concatenate([embedding_1, embedding_2])
outputs = keras.layers.Dense(1)(encoded_inputs)
model = keras.models.Model(inputs=[input_1, input_2], outputs=[outputs])

model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam())
model.fit((one_hot_1, one_hot_2), y_str, epochs=1)

ValueError: Data cardinality is ambiguous:
  x sizes: 2
  y sizes: 10374841
Make sure all arrays contain the same number of samples.

In [102]:
import numpy as np
a = np.array([[1,4],[3,1]])
a

array([[1, 4],
       [3, 1]])

In [105]:
np.random.shuffle(a)
a

array([[1, 4],
       [3, 1]])

In [94]:
a

array([[1, 4],
       [1, 3]])

In [107]:
df = pd.read_pickle('distances.pkl')
df['distance_meters'] = df['distance'] * 1000
# n_values = len(set(df.GEOID_x))
n_values = max(df.GEOID_x) + 1
dataset = df[['GEOID_x', 'GEOID_y', 'distance_meters']].astype('int32').to_numpy()
np.random.shuffle(dataset)
X = dataset[:,:2]
y = dataset[:,-1]
embedding_dim = 2
input_1 = keras.layers.Input(shape=[], dtype=tf.int32)
input_2 = keras.layers.Input(shape=[], dtype=tf.int32)
embedding_1 = keras.layers.Embedding(input_dim=n_values, output_dim=embedding_dim)(input_1)
embedding_2 = keras.layers.Embedding(input_dim=n_values, output_dim=embedding_dim)(input_2)
encoded_inputs = keras.layers.concatenate([embedding_1, embedding_2])
outputs = keras.layers.Dense(1)(encoded_inputs)
model = keras.models.Model(inputs=[input_1, input_2], outputs=[outputs])

model.compile(loss='mae', optimizer=keras.optimizers.SGD())
history = model.fit((X[:,0], X[:,1]), y, epochs=32)

Epoch 1/32
324214/324214 [==============================] - 348s 1ms/step - loss: 644097.9375
Epoch 2/32
324214/324214 [==============================] - 338s 1ms/step - loss: 464445.8750
Epoch 3/32
324214/324214 [==============================] - 369s 1ms/step - loss: 463171.9062
Epoch 4/32
324214/324214 [==============================] - 351s 1ms/step - loss: 462437.1875
Epoch 5/32
324214/324214 [==============================] - 348s 1ms/step - loss: 461918.0000
Epoch 6/32
324214/324214 [==============================] - 350s 1ms/step - loss: 461582.2812
Epoch 7/32
324214/324214 [==============================] - 336s 1ms/step - loss: 461325.7188
Epoch 8/32
324214/324214 [==============================] - 474s 1ms/step - loss: 461100.7500
Epoch 9/32
324214/324214 [==============================] - 448s 1ms/step - loss: 460928.5938
Epoch 10/32
324214/324214 [==============================] - 323s 997us/step - loss: 460757.6250
Epoch 11/32
324214/324214 [==============================

In [138]:
df = pd.read_pickle('distances.pkl')
df['distance_norm'] = (df['distance'] - df['distance'].mean())/df['distance'].std() 
n_values = max(df.GEOID_x) + 1
dataset = df[['GEOID_x', 'GEOID_y', 'distance_norm']].astype('int32').to_numpy()
# dataset = np.repeat(dataset, 5, axis=0)
np.random.shuffle(dataset)
X = dataset[:,:2]
y = dataset[:,-1]
embedding_dim = 2
input_1 = keras.layers.Input(shape=[], dtype=tf.int32)
input_2 = keras.layers.Input(shape=[], dtype=tf.int32)
embedding_1 = keras.layers.Embedding(input_dim=n_values, input_length=1, output_dim=embedding_dim)(input_1)
embedding_2 = keras.layers.Embedding(input_dim=n_values, input_length=1, output_dim=embedding_dim)(input_2)
encoded_inputs = keras.layers.concatenate([embedding_1, embedding_2])
outputs = keras.layers.Dense(1)(encoded_inputs)
model3 = keras.models.Model(inputs=[input_1, input_2], outputs=[outputs])

model3.compile(loss='mae', optimizer=keras.optimizers.Adam())
history = model3.fit((X[:,0], X[:,1]), y, epochs=2, batch_size=1000)

Epoch 1/2
51875/51875 [==============================] - 121s 2ms/step - loss: 0.2360
Epoch 2/2
51875/51875 [==============================] - 113s 2ms/step - loss: 0.2331


In [160]:
df = pd.read_pickle('distances.pkl')
df['distance_norm'] = (df['distance'] - df['distance'].mean())/df['distance'].std() 
n_values = max(df.GEOID_x) + 1
dataset = df[['GEOID_x', 'GEOID_y', 'distance_norm']].astype('int32').to_numpy()
dataset = np.repeat(dataset, 5, axis=0)
np.random.shuffle(dataset)
X = dataset[:,:2]
y = dataset[:,-1]
embedding_dim = 10
input_1 = keras.layers.Input(shape=X.shape[1:], dtype=tf.int32)
# embedding_1 = keras.layers.Embedding(input_dim=n_values, input_length=1, output_dim=embedding_dim)(input_1)
embedding_1 = keras.layers.Dense(embedding_dim)(input_1)
# flattened = keras.layers.Flatten()(embedding_1)
outputs = keras.layers.Dense(1)(embedding_1)
model3 = keras.models.Model(inputs=[input_1], outputs=[outputs])

model3.compile(loss='mae', optimizer=keras.optimizers.Adam())
history = model3.fit(X, y, epochs=16, batch_size=1000)

Epoch 1/16
51875/51875 [==============================] - 51s 888us/step - loss: 50.6865
Epoch 2/16
51875/51875 [==============================] - 51s 982us/step - loss: 28.6947s - loss: 2
Epoch 3/16
51875/51875 [==============================] - 54s 1ms/step - loss: 26.5745
Epoch 4/16
51875/51875 [==============================] - 63s 1ms/step - loss: 25.3324
Epoch 5/16
51875/51875 [==============================] - 47s 913us/step - loss: 23.9373
Epoch 6/16
51875/51875 [==============================] - 46s 880us/step - loss: 20.2047
Epoch 7/16
51875/51875 [==============================] - 52s 996us/step - loss: 19.0208
Epoch 8/16
51875/51875 [==============================] - 61s 1ms/step - loss: 17.4131
Epoch 9/16
51875/51875 [==============================] - 52s 995us/step - loss: 16.3530
Epoch 10/16
51875/51875 [==============================] - 46s 893us/step - loss: 15.4113
Epoch 11/16
51875/51875 [==============================] - 59s 1ms/step - loss: 13.9960
Epoch 12/16
5187

In [108]:
import dsci592.covid_collector as cc
cc.get_fips_from_county('Camden_NewJersey_UnitedStates')

34007

In [161]:
for layer in model3.layers:
    print(layer.name, layer)

input_80 <keras.engine.input_layer.InputLayer object at 0x0000027D970B0DC0>
dense_42 <keras.layers.core.dense.Dense object at 0x0000027DA75785B0>
dense_43 <keras.layers.core.dense.Dense object at 0x0000027D972EC580>


In [166]:
embeddings = model3.layers[1].weights[0]
embeddings

<tf.Variable 'dense_42/kernel:0' shape=(2, 10) dtype=float32, numpy=
array([[ 1.91415718e-04, -4.10010636e-01,  2.46048352e-04,
        -2.71979123e-01, -3.93924665e-06,  4.75068373e-04,
         1.38544021e-02, -5.57864842e-04,  1.15778275e-04,
         3.20300072e-01],
       [ 7.71568215e-04,  4.01340067e-01, -3.14847392e-04,
        -2.12679118e-01, -4.42936152e-06, -6.48852438e-04,
         2.35119700e-01, -4.91024577e-04, -1.19434728e-04,
         3.10401708e-01]], dtype=float32)>

In [113]:
def cosine_similarity(u, v):
    """
    Cosine similarity reflects the degree of similariy between u and v
        
    Arguments:
        u -- a word vector of shape (n,)          
        v -- a word vector of shape (n,)

    Returns:
        cosine_similarity -- the cosine similarity between u and v defined by the formula above.
    """
    
    distance = 0.0
    
    ### START CODE HERE ###
    # Compute the dot product between u and v (≈1 line)
    dot = np.dot(u, v)
    # Compute the L2 norm of u (≈1 line)
    norm_u = np.sqrt(np.sum(u**2))
    
    # Compute the L2 norm of v (≈1 line)
    norm_v = np.sqrt(np.sum(v**2))
    # Compute the cosine similarity defined by formula (1) (≈1 line)
    cosine_similarity = dot / np.dot(norm_u, norm_v)
    ### END CODE HERE ###
    
    return cosine_similarity

In [126]:
def complete_analogy(word_a, word_b, word_c, embedding, fips_to_name, name_to_fips):
    """
    Performs the word analogy task as explained above: a is to b as c is to ____. 
    
    Arguments:
    word_a -- a word, string
    word_b -- a word, string
    word_c -- a word, string
    word_to_vec_map -- dictionary that maps words to their corresponding vectors. 
    
    Returns:
    best_word --  the word such that v_b - v_a is close to v_best_word - v_c, as measured by cosine similarity
    """
    n_a, n_b, n_c = name_to_fips(word_a), name_to_fips(word_b), name_to_fips(word_c)
    e_a, e_b, e_c = embedding[n_a], embedding[n_b], embedding[n_c]
    
    max_cosine_sim = -100              # Initialize max_cosine_sim to a large negative number
    best_word = None                   # Initialize best_word with None, it will help keep track of the word to output

    # loop over the whole word vector set
    for i in range(embeddings.shape[0]):        
        # to avoid best_word being one of the input words, pass on them.
        if i in [n_a, n_b, n_c] :
            continue
        try:
            fips_to_name(i)
        except:
            continue
        
        ### START CODE HERE ###
        # Compute cosine similarity between the vector (e_b - e_a) and the vector ((w's vector representation) - e_c)  (≈1 line)
        cosine_sim = cosine_similarity((e_b - e_a), (embedding[i] - e_c))
        
        # If the cosine_sim is more than the max_cosine_sim seen so far,
            # then: set the new max_cosine_sim to the current cosine_sim and the best_word to the current word (≈3 lines)
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            best_word = i
        ### END CODE HERE ###
        
    return fips_to_name(best_word)

In [ ]:
complete_analogy('Camden_NewJersey_UnitedStates', 'Salem_NewJersey_UnitedStates', 'Morristown_NewJersey_UnitedStates',embeddings,cc.get_county_from_fips, cc.get_fips_from_county)


72154

In [134]:
x = np.array([[1,2],[3,4]])

np.repeat(x,2,axis=0)

array([[1, 2],
       [1, 2],
       [3, 4],
       [3, 4]])